<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=810168dec4a31f3850132862a8bce7170ae415973fa551862496e4e87e382236
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-08 14:08:52
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -44.41 K (-0.31%)
Current PnL: -20.98 L (-13.92%)
CY Booked + Current PnL: -9.54 L (-6.33%)
-------------------
Total profit:  1.44 L
Total loss:  -22.42 L
-------------------
Total Booked + Current PnL: 17.44 L (14.1%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.49 L (62.93%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.21,119128.0,18624.0,10483.0,3.55,18.53,8.80,28.96,79.0,1.78,0.85,58.79,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.19,83224.0,-17553.0,17643.0,-0.17,-17.42,21.20,0.09,245.0,-0.99,0.59,9.34,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,49.0,H-MC,3.06,189610.0,24940.0,18392.0,-0.84,15.15,9.70,26.31,89.0,1.36,1.35,24.34,X40N,NTT,AC
19,CIPLA,1662.74,-22.87,40.0,H-LC,3.81,205760.0,1260.0,22037.0,-0.75,0.62,10.71,11.39,10.0,0.06,1.47,9.62,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.33,216029.0,5663.0,23115.0,0.71,2.69,10.70,13.68,37.0,0.24,1.54,19.66,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3813.51,59.0,M-SC,6.47,175712.0,-21420.0,111243.0,7.29,-10.87,63.31,45.57,236.0,-0.19,1.25,24.58,XY24,NTT,AUTO
37,INDIAMART,4810.62,-53.71,48.0,H-SC,9.14,129156.0,5820.0,125811.0,3.85,4.72,97.41,106.72,119.0,0.05,0.92,29.20,AR,ATH,MISC
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.21,119128.0,18624.0,10483.0,3.55,18.53,8.80,28.96,79.0,1.78,0.85,58.79,MH,ATH,METALS
40,INFY,2275.00,-18.52,53.0,H-LC,8.00,317604.0,4716.0,166964.0,2.24,1.51,52.57,54.87,3.0,0.03,2.27,8.26,X40,BTT,IT
48,LTIM,7230.20,-2.60,59.0,H-LC,2.46,241650.0,-4501.0,83708.0,1.92,-1.83,34.64,32.18,16.0,-0.05,1.73,33.56,X200,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.56,27.0,M-SC,37.78,52801.0,-12205.0,163134.0,-2.98,-18.77,308.96,232.18,198.0,-0.07,0.38,0.00,XY24,NTT,MISC
55,RAJOOENG,143.10,-42.59,38.0,H-SC,15.99,86970.0,-15530.0,56130.0,-2.84,-15.15,64.54,39.61,114.0,-0.28,0.62,6.22,AR,ATH,MISC
75,TITAGARH,1548.00,-3.85,52.0,H-SC,4.73,205610.0,-36567.0,147340.0,-2.74,-15.10,71.66,45.74,158.0,-0.25,1.47,33.73,XY24,NTT,ENGINEERING
85,WHIRLPOOL,2270.00,-48.24,27.0,M-SC,1.60,87518.0,-3980.0,82731.0,-2.22,-4.35,94.53,86.07,223.0,-0.05,0.62,27.09,XR,NTT,DURABLES
27,HAPPSTMNDS,1488.71,-27.75,14.0,H-SC,12.47,77976.0,-50189.0,160217.0,-2.20,-39.16,205.47,85.85,132.0,-0.31,0.56,0.00,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.0,43.54,49.0,M-MC,12.46,226391.0,1429.0,166216.0,-1.06,0.64,73.42,74.53,192.0,0.01,1.62,33.09,XY24,BTT,STEEL
35,ICICIPRULI,790.0,-21.80,45.0,H-MC,1.90,134651.0,-1137.0,43883.0,-1.28,-0.84,32.59,31.48,107.0,-0.03,0.96,11.18,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.79,48.0,M-LC,5.86,151664.0,719.0,108728.0,0.45,0.48,71.69,72.51,53.0,0.01,1.08,6.53,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,60.0,H-SC,13.44,129411.0,-13068.0,32586.0,-1.01,-9.17,25.18,13.70,163.0,-0.40,0.92,52.74,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.19,83224.0,-17553.0,17643.0,-0.17,-17.42,21.20,0.09,245.0,-0.99,0.59,9.34,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-68.16,46.0,H-SC,1.99,219879.0,-49788.0,86127.0,-0.88,-18.46,39.17,13.47,138.0,-0.58,1.57,11.80,XY24,NTT,PAINTS
18,CERA,9475.0,-23.15,37.0,H-SC,1.59,140300.0,-35603.0,77628.0,0.18,-20.24,55.33,23.89,149.0,-0.46,1.00,21.17,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-23.04,50.0,H-MC,6.43,104215.0,-27620.0,69845.0,1.77,-20.95,67.02,32.03,98.0,-0.40,0.74,16.73,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,49.0,M-MC,12.46,226391.0,1429.0,166216.0,-1.06,0.64,73.42,74.53,192.0,0.01,1.62,33.09,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213868.0,8792.0,103769.0,0.36,4.29,48.52,54.89,52.0,0.08,1.53,29.42,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.78,49.0,H-MC,3.06,189610.0,24940.0,18392.0,-0.84,15.15,9.70,26.31,89.0,1.36,1.35,24.34,X40N,NTT,AC
84,VOLTAS,1530.00,-2.06,49.0,H-MC,1.29,205515.0,13773.0,23984.0,-0.15,7.18,11.67,19.69,99.0,0.57,1.47,14.47,XY25,NTT,AC
17,CAMS,5211.76,-6.94,42.0,H-SC,1.74,105834.0,3830.0,40090.0,-0.55,3.75,37.88,43.06,122.0,0.10,0.76,21.51,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.21,119128.0,18624.0,10483.0,3.55,18.53,8.80,28.96,79.0,1.78,0.85,58.79,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.18,192881.0,13073.0,99334.0,-0.66,7.27,51.50,62.52,88.0,0.13,1.38,43.84,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213868.0,8792.0,103769.0,0.36,4.29,48.52,54.89,52.0,0.08,1.53,29.42,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.71,48.0,H-SC,9.14,129156.0,5820.0,125811.0,3.85,4.72,97.41,106.72,119.0,0.05,0.92,29.20,AR,ATH,MISC
86,WIPRO,420.00,-13.79,48.0,M-LC,5.86,151664.0,719.0,108728.0,0.45,0.48,71.69,72.51,53.0,0.01,1.08,6.53,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.19,276637.0,-39205.0,151874.0,-1.45,-12.41,54.90,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-19.07,29.0,H-LC,5.70,204406.0,-47362.0,100568.0,-1.28,-18.81,49.20,21.13,27.0,-0.47,1.46,9.95,X40,ATH,PAINTS
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
19,CIPLA,1662.74,-22.87,40.0,H-LC,3.81,205760.0,1260.0,22037.0,-0.75,0.62,10.71,11.39,10.0,0.06,1.47,9.62,X40N,ATH,PHARMA
31,HINDUNILVR,2922.00,-10.50,41.0,H-LC,8.01,251340.0,-5723.0,40868.0,-0.15,-2.23,16.26,13.67,24.0,-0.14,1.79,17.73,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,0.39,54.0,H-LC,1.97,161415.0,-24680.0,72153.0,-0.69,-13.26,44.70,25.51,15.0,-0.34,1.15,18.95,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
48,LTIM,7230.2,-2.60,59.0,H-LC,2.46,241650.0,-4501.0,83708.0,1.92,-1.83,34.64,32.18,16.0,-0.05,1.73,33.56,X200,ATH,IT
51,NESTLEIND,1377.0,-10.39,50.0,H-LC,2.77,275652.0,10226.0,46558.0,0.10,3.85,16.89,21.40,11.0,0.22,1.97,10.75,XY25,NTT,FMCG
57,RELIANCE,1533.0,-13.66,51.0,H-LC,3.33,216029.0,5663.0,23115.0,0.71,2.69,10.70,13.68,37.0,0.24,1.54,19.66,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-23.20,28.0,H-LC,10.19,276637.0,-39205.0,151874.0,-1.45,-12.41,54.90,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
74,TCS,4406.34,-28.69,51.0,H-LC,12.18,284312.0,-61644.0,129874.0,1.71,-17.82,45.68,19.73,1.0,-0.47,2.03,4.72,X40,ATH,IT
80,UNITDSPR,1644.00,-12.23,52.0,H-LC,6.77,229964.0,-4180.0,52800.0,-1.09,-1.79,22.96,20.77,86.0,-0.08,1.64,5.01,X40N,NTT,BREWERIES
1,ABB,7934.00,-40.94,41.0,H-LC,7.52,246936.0,-14683.0,133889.0,-1.44,-5.61,54.22,45.57,7.0,-0.11,1.76,5.07,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.39,54.0,H-LC,1.97,161415.0,-24680.0,72153.0,-0.69,-13.26,44.70,25.51,15.0,-0.34,1.15,18.95,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-19.07,29.0,H-LC,5.70,204406.0,-47362.0,100568.0,-1.28,-18.81,49.20,21.13,27.0,-0.47,1.46,9.95,X40,ATH,PAINTS
19,CIPLA,1662.74,-22.87,40.0,H-LC,3.81,205760.0,1260.0,22037.0,-0.75,0.62,10.71,11.39,10.0,0.06,1.47,9.62,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-13.66,51.0,H-LC,3.33,216029.0,5663.0,23115.0,0.71,2.69,10.70,13.68,37.0,0.24,1.54,19.66,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,51.0,H-LC,12.18,284312.0,-61644.0,129874.0,1.71,-17.82,45.68,19.73,1.0,-0.47,2.03,4.72,X40,ATH,IT
40,INFY,2275.00,-18.52,53.0,H-LC,8.00,317604.0,4716.0,166964.0,2.24,1.51,52.57,54.87,3.0,0.03,2.27,8.26,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,H-LC,2.33,195160.0,-4978.0,25859.0,-0.18,-2.49,13.25,10.44,4.0,-0.19,1.39,2.94,X40,NTT,FMCG
83,VBL,671.64,-23.20,28.0,H-LC,10.19,276637.0,-39205.0,151874.0,-1.45,-12.41,54.90,35.67,5.0,-0.26,1.98,0.00,X40N,ATH,FMCG
1,ABB,7934.00,-40.94,41.0,H-LC,7.52,246936.0,-14683.0,133889.0,-1.44,-5.61,54.22,45.57,7.0,-0.11,1.76,5.07,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,43.0,L-SC,30.78,78701.0,-34848.0,74900.0,-0.94,-30.69,95.17,35.27,268.0,-0.47,0.56,92.62,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,44.0,H-SC,10.87,91575.0,-6405.0,28004.0,-1.29,-6.54,30.58,22.05,152.0,-0.23,0.65,33.11,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,69.0,H-MC,14.21,119128.0,18624.0,10483.0,3.55,18.53,8.80,28.96,79.0,1.78,0.85,58.79,MH,ATH,METALS
37,INDIAMART,4810.62,-53.71,48.0,H-SC,9.14,129156.0,5820.0,125811.0,3.85,4.72,97.41,106.72,119.0,0.05,0.92,29.20,AR,ATH,MISC
78,UJJIVANSFB,60.00,105.37,60.0,H-SC,13.44,129411.0,-13068.0,32586.0,-1.01,-9.17,25.18,13.70,163.0,-0.40,0.92,52.74,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,60.09,61.0,H-SC,7.36,147900.0,-31171.0,96904.0,-0.18,-17.41,65.52,36.71,135.0,-0.32,1.06,46.75,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,64.0,M-SC,13.47,143721.0,-5499.0,149685.0,-2.08,-3.69,104.15,96.62,208.0,-0.04,1.03,55.54,XY25,NTT,FINANCE
26,GREENPANEL,537.00,233.33,60.0,M-SC,5.08,148864.0,-34214.0,113196.0,-0.31,-18.69,76.04,43.14,230.0,-0.30,1.06,40.03,XY24,NTT,MISC
32,HINDZINC,730.22,31.07,68.0,M-LC,6.90,213868.0,8792.0,103769.0,0.36,4.29,48.52,54.89,52.0,0.08,1.53,29.42,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,66.0,H-MC,11.18,192881.0,13073.0,99334.0,-0.66,7.27,51.50,62.52,88.0,0.13,1.38,43.84,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.42
2,50,75.12


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.07
LC    32.19
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.03
X40      14.66
X40N     11.17
XR       10.91
XY25     10.39
AR        8.14
OX40N     7.67
X200      1.73
MH        1.62
X5K       1.53
SR        1.13
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.98
H-LC    25.69
H-MC    20.48
M-SC    13.68
M-LC     5.49
M-MC     2.94
L-SC     1.41
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.72,-6.81,42.01
IT,12.80,-17.38,80.96
FINANCE,10.55,-14.20,69.35
MISC,6.90,-18.84,84.69
BANKS,6.30,-13.00,72.45
PAINTS,5.49,-20.01,37.90
ELECTRICAL,5.36,-9.61,48.06
INSURANCE,3.84,-2.89,41.35
AUTO,3.54,-15.76,71.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3228537.0,22
XR,1380953.0,14
AR,1189582.0,9
X40,787288.0,10
X40N,672984.0,9
OX40N,568568.0,10
XY25,482117.0,7
SR,263686.0,2
X5K,103769.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3734238.0,29
M-SC,1515518.0,17
H-MC,1284538.0,15
H-LC,1196228.0,15
M-LC,380477.0,4
M-MC,371721.0,2
L-SC,263128.0,3
L-MC,60204.0,1
L-LC,42672.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1197534.0      6
M-SC       XY24       844341.0      7
H-SC       AR         813652.0      5
           XR         807015.0      7
H-MC       XY24       577426.0      4
H-LC       X40        499691.0      5
M-MC       XY24       371721.0      2
H-SC       X40N       311856.0      4
           SR         263686.0      2
           OX40N      263446.0      4
H-LC       X40N       226711.0      3
H-MC       X40        213814.0      4
H-LC       AR         206042.0      2
H-MC       XY25       179219.0      2
L-SC       XR         174086.0      2
M-SC       AR         169888.0      2
M-LC       XY24       167980.0      2
M-SC       XY25       149685.0      1
           OX40N      146235.0      4
H-MC       X40N       134417.0      2
M-SC       XR         131586.0      2
H-LC       XY25       110541.0      3
M-LC       XR         108728.0      1
           X5K        103769.0      1
H-MC       XR          99334.0      1
L-SC       OX40N       89042.0      1
H-LC       X200        83708.0      1
H-SC       MH          77049.0      1
M-SC       X40         73783.0      1
H-MC       OX40N       69845.0      1
H-LC       XY24        69535.0      1
L-MC       XR          60204.0      1
L-LC       XY25        42672.0      1
H-MC       MH          10483.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
